In [13]:
import pandas as pd

data = pd.read_parquet('../cache/encoded_scaled.parquet')
data = data.drop(columns=['Impact_bin'])
data.head()

,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,pct_change_15min,pct_change_30min,pct_change_24h
Date,,,,,,,,,,,,,,,,,,
2019-01-01,-1.530752,218,247,4,218,247,32,249,7,0.605751,0.889228,2.004333,1.164239,0.375252,0.714246,-0.033061,-0.226363,-2.433464
2019-01-01,-1.530752,27,30,32,218,247,32,30,10,0.731167,0.367070,0.597142,0.468568,-0.716820,0.425568,-0.033061,-0.226363,-2.433464
2019-01-01,-1.530752,218,228,9,218,227,9,229,1,0.563945,4.022178,1.300737,1.004049,0.960180,0.848379,-0.033061,-0.226363,-2.433464
2019-01-01,-1.530752,204,228,32,218,247,32,229,8,-0.523000,0.367070,-0.036094,-0.520616,0.334805,-0.355905,-0.033061,-0.226363,-2.433464
2019-01-01,-1.530752,204,228,6,218,247,32,229,4,0.292209,1.933545,-0.247173,0.921836,-0.100779,0.329342,-0.033061,-0.226363,-2.433464


In [14]:
data.dtypes

Timestamp           float64
Actor1Country         int32
Actor1GeoCountry      int32
Actor1Type            int32
Actor2Country         int32
Actor2GeoCountry      int32
Actor2Type            int32
ActionCountry         int32
EventType             int32
GoldsteinScale      float64
NumSources          float64
NumArticles         float64
AvgTone             float64
Magnitude           float64
Impact              float64
pct_change_15min    float64
pct_change_30min    float64
pct_change_24h      float64
dtype: object

In [15]:
X = data.drop(columns=['pct_change_15min', 'pct_change_30min', 'pct_change_24h'])
y = data['pct_change_30min']
sample = data.tail(10000)
sample = sample.drop(columns=['pct_change_15min', 'pct_change_24h', 'Timestamp'])
sample.head()

,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,pct_change_30min
Date,,,,,,,,,,,,,,,
2023-12-26 13:45:00,218,102,9,218,102,30,103,7,0.605751,-0.155088,-0.106454,-1.176927,0.953957,0.897950,-0.243201
2023-12-26 13:45:00,218,183,29,218,247,32,184,13,1.149223,-0.155088,-0.106454,1.318563,-0.215898,0.900866,-0.243201
2023-12-26 13:45:00,218,228,6,218,247,32,229,10,0.731167,-0.677247,-0.106454,0.627611,-1.202186,0.241863,-0.243201
2023-12-26 13:45:00,93,102,9,218,247,32,103,5,-1.150084,-0.677247,-0.669330,-0.198623,-0.505251,-0.551273,-0.243201
2023-12-26 13:45:00,218,247,32,218,227,16,229,8,-0.523000,-0.677247,-0.669330,-0.183876,-0.520807,-0.195528,-0.243201


In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Quantifies multicollinearity
vif_data = pd.DataFrame()
vif_data["Variable"] = sample.columns
vif_data["VIF"] = [variance_inflation_factor(sample.values, i) for i in range(sample.shape[1])]
print(vif_data)

            Variable        VIF
0      Actor1Country   9.832815
1   Actor1GeoCountry  18.932804
2         Actor1Type   4.146340
3      Actor2Country  14.195148
4   Actor2GeoCountry  19.923738
5         Actor2Type   7.091387
6      ActionCountry  19.274380
7          EventType   3.107855
8     GoldsteinScale   5.379483
9         NumSources   1.456379
10       NumArticles   1.358373
11           AvgTone   1.976615
12         Magnitude   2.208775
13            Impact   5.701576
14  pct_change_30min   1.003786


### dynamic factor model (DFM)

In [18]:
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

# Fit a dynamic factor model
model = DynamicFactor(sample, k_factors=1, factor_order=1)
results = model.fit()
print(results.summary())

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                                                                                                                       Statespace Model Results                                                                                                                                      
Dep. Variable:     ['Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type', 'ActionCountry', 'EventType', 'GoldsteinScale', 'NumSources', 'NumArticles', 'AvgTone', 'Magnitude', 'Impact', 'pct_change_30min']   No. Observations:                10000
Model:                                                                                                                                                                                                                     DynamicFactor(factors=1, order=1)   Log Likelihood             -611606.952
Date:                                                                                                                 

In [19]:
forecast_steps = 5
forecast = results.get_forecast(steps=forecast_steps)
# Extract the predicted values from the forecast result
predicted_values = forecast.predicted_mean
# Or for the full prediction including uncertainty (confidence intervals)
prediction_conf_int = forecast.conf_int()

# Display the predicted values and the confidence intervals
print("Predicted Values:")
print(predicted_values)

print("\nConfidence Intervals for Predictions:")
print(prediction_conf_int)

Predicted Values:
       Actor1Country  Actor1GeoCountry  Actor1Type  Actor2Country  \
10000      64.992077        139.531487   -2.975143      54.822149   
10001      56.469912        121.235251   -2.585023      47.633528   
10002      49.065226        105.338132   -2.246059      41.387524   
10003      42.631489         91.525541   -1.951542      35.960535   
10004      37.041383         79.524144   -1.695644      31.245166   

       Actor2GeoCountry  Actor2Type  ActionCountry  EventType  GoldsteinScale  \
10000        126.260212    1.729952     178.852413   4.290548       -3.994538   
10001        109.704188    1.503110     155.400173   3.727945       -3.470749   
10002         95.319094    1.306013     135.023137   3.239114       -3.015643   
10003         82.820262    1.134760     117.318064   2.814381       -2.620213   
10004         71.960354    0.985963     101.934591   2.445342       -2.276635   

       NumSources  NumArticles   AvgTone  Magnitude    Impact  \
10000   -0.3884

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


### State-Space Model

In [20]:
sample.columns

Index(['Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor2Country',
       'Actor2GeoCountry', 'Actor2Type', 'ActionCountry', 'EventType',
       'GoldsteinScale', 'NumSources', 'NumArticles', 'AvgTone', 'Magnitude',
       'Impact', 'pct_change_30min'],
      dtype='object')

In [21]:
from statsmodels.tsa.statespace.varmax import VARMAX

# Check for missing values
sample = sample.fillna(sample.mean())  # Impute missing values with mean (or other strategy)

# Select the features and target columns you want to use for forecasting
features = sample[['Actor1Country', 'Actor2Country', 'GoldsteinScale', 'NumSources', 'NumArticles']]
target = sample['pct_change_30min']  # Example target column

# Fit a VARMAX model
model = VARMAX(features, order=(1, 1))  # order of the model (lags and error terms)
model_fitted = model.fit(disp=False)


C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [22]:
# Make predictions
forecast_steps = 10  # number of steps to forecast
forecast = model_fitted.forecast(steps=forecast_steps)
print(forecast)

       Actor1Country  Actor2Country  GoldsteinScale  NumSources  NumArticles
10000     177.061860     188.909285       -0.157351   -0.089884    -0.074348
10001     176.058012     186.307871       -0.126137   -0.070360    -0.045027
10002     175.980713     186.153959       -0.124765   -0.067105    -0.043638
10003     175.975653     186.140639       -0.124698   -0.066577    -0.043493
10004     175.975366     186.139135       -0.124694   -0.066494    -0.043472
10005     175.975355     186.138934       -0.124694   -0.066481    -0.043468
10006     175.975356     186.138904       -0.124693   -0.066479    -0.043468
10007     175.975356     186.138900       -0.124693   -0.066478    -0.043468
10008     175.975356     186.138899       -0.124693   -0.066478    -0.043468
10009     175.975356     186.138899       -0.124693   -0.066478    -0.043468


C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


### Multivariate Exponential Smoothing

### Canonical Correlation Analysis (CCA)

### 11